In [1]:
import pandas as pd
import os
import re
import time
import numpy as np
import statistics
from natsort import natsorted, ns
import random
from sklearn.preprocessing import StandardScaler 

from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from xgboost import XGBClassifier

In [10]:
def spatial_two (Node_class, Adj,n):
    F_vec=[]
    for i in range(n):
        print("\rProcessing file {} ({}%)".format(i, 100*i//(n-1)), end='', flush=True)
        node_F=[]
        list_out=[]
        list_In=[]
        S_nbd_out=[]
        S_nbd_in=[]
        for j in range(n):
            if Adj[i][j]==1 and i!=j:
                list_out.append(Adj[j][j])
                for k in range(n):
                    if A[j][k]==1 and k!=j:
                        S_nbd_out.append(Adj[k][k])
            if Adj[j][i]==1 and i!=j:
                list_In.append(A[j][j])
                for k in range(n):
                    if Adj[k][j]==1 and k!=j:
                        S_nbd_in.append(Adj[k][k])

        for d in Node_class:
            count=0
            count_in=0
            for k in range(len(list_out)):
                if list_out[k]==d:
                    count=count+1
            node_F.append(count)
            for k in range(len(list_In)):
                if list_In[k]==d:
                    count_in=count_in+1
            node_F.append(count_in)
        for d in Node_class:
            count_S_out=0
            count_S_in=0
            for k in range(len(S_nbd_out)):
                if S_nbd_out[k]==d:
                    count_S_out=count_S_out+1
            node_F.append(count_S_out)
            for k in range(len(S_nbd_in)):
                if S_nbd_in[k]==d:
                    count_S_in=count_S_in+1
            node_F.append(count_S_in)
        F_vec.append(node_F)
    return F_vec

In [257]:
import pandas as pd
import numpy as np
# read the dataset from the link, extract and make a dataframe
import requests, zipfile, io
r = requests.get('https://linqs-data.soe.ucsc.edu/public/datasets/cora/cora.zip', stream=True)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()
feature_names = ["w_{}".format(ii) for ii in range(1433)]
column_names = ["ID"]+ feature_names + ["class"]
node_data = pd.read_csv("cora/cora.content", sep='\t', header=None, names=column_names)
node_data.head()

,ID,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,...,w_1424,w_1425,w_1426,w_1427,w_1428,w_1429,w_1430,w_1431,w_1432,class
0,31336,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Neural_Networks
1,1061127,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Rule_Learning
2,1106406,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
3,13195,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
4,37879,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Probabilistic_Methods


In [3]:
DataS=node_data.replace({'class': {'Case_Based':0 , 'Genetic_Algorithms': 1,'Neural_Networks':2,'Probabilistic_Methods':3 , 'Reinforcement_Learning': 4,'Rule_Learning':5,'Theory':6}})
# Case_Based==0
#		Genetic_Algorithms==1
#		Neural_Networks==2
#		Probabilistic_Methods==3
#		Reinforcement_Learning==4
#		Rule_Learning==5
#		Theory==6
DataS.head()

,ID,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,...,w_1424,w_1425,w_1426,w_1427,w_1428,w_1429,w_1430,w_1431,w_1432,class
0,31336,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
1,1061127,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,5
2,1106406,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,13195,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
4,37879,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [4]:
Data=DataS.sort_values(by=['ID'],ascending=True)
Data.head()

,ID,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,...,w_1424,w_1425,w_1426,w_1427,w_1428,w_1429,w_1430,w_1431,w_1432,class
163,35,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
168,40,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
552,114,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,4
1459,117,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
554,128,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [5]:
catagories=Data['class'].to_numpy()
data_by_class = {cls: Data.loc[Data['class'] == cls].drop(['class', 'ID'], axis=1) for cls in range(max(catagories) + 1)}
basis = [[max(df[f'w_{i}']) for i in range(len(df.columns))] for df in data_by_class.values()]
sel_basis = [[int(list(df[f'w_{i}'].to_numpy()).count(1) >= int(len(df[f'w_{i}'].index)*0.1)) 
              for i in range(len(df.columns))]
             for df in data_by_class.values()]
IdS=list(DataS['ID'].to_numpy())
Id=list(Data['ID'].to_numpy())

In [6]:
Fec=[]
for i in range(2708):
    vec=[]
    f=Data.loc[IdS.index(Id[i]), feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    count5=0
    count6=0
    for j in range(1433):
        if f[j]==1 and basis[0][j]==1:
            count=count+1;
        if f[j]==1 and basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and basis[4][j]==1:
            count4=count4+1;
        if f[j]==1 and basis[5][j]==1:
            count5=count5+1;
        if f[j]==1 and basis[6][j]==1:
            count6=count6+1;
    vec.append(count)
    vec.append(count1)
    vec.append(count2)
    vec.append(count3)
    vec.append(count4)
    vec.append(count5)
    vec.append(count6)
    #print(f)
    f.clear()
    Fec.append(vec)

In [7]:
SFec=[]
for i in range(2708):
    Svec=[]
    f=Data.loc[IdS.index(Id[i]), feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    count5=0
    count6=0
    for j in range(1433):
        if f[j]==1 and sel_basis[0][j]==1:
            count=count+1;
        if f[j]==1 and sel_basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and sel_basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and sel_basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and sel_basis[4][j]==1:
            count4=count4+1;
        if f[j]==1 and sel_basis[5][j]==1:
            count5=count5+1;
        if f[j]==1 and sel_basis[6][j]==1:
            count6=count6+1;
    Svec.append(count)
    Svec.append(count1)
    Svec.append(count2)
    Svec.append(count3)
    Svec.append(count4)
    Svec.append(count5)
    Svec.append(count6)
    #print(f)
    f.clear()
    SFec.append(Svec)

In [8]:
z =np.array(SFec)
data2=pd.DataFrame({"S_0":z[:,0]})
for i in range(6):
    data2.insert(loc=i+1, column="S_{}".format(i+1), value=z[:,i+1])
#data2.insert(loc=7,column='Class',value=Data['subject'].to_numpy())
data2.head()

,S_0,S_1,S_2,S_3,S_4,S_5,S_6
0,1,7,3,2,3,2,1
1,6,5,3,4,6,7,4
2,1,3,1,1,4,3,1
3,5,4,4,4,8,2,5
4,5,3,3,2,8,2,3


In [26]:
Fe=[]
for i in range(l):
    Fe.append("{}".format(i))
y =np.array(Fec)
data1 = pd.DataFrame(y, columns =Fe)
data1.head()

,0,1,2,3,4,5,6
0,16,20,20,16,17,12,14
1,16,18,18,17,15,16,15
2,10,11,12,10,12,7,8
3,13,17,19,18,20,14,18
4,17,17,19,17,20,15,17


In [258]:
p_data = open("cora/cora.cites")


# Create a list from the data
myls=list(p_data)

# Remove the new line symbole from the list
mylist = [line.rstrip('\n') for line in myls]
#print(mylist)
#print(graph_ind)
#print(graph_level)

p_data.close()
mynode=[]
i=0
for d in mylist:
    d=mylist[i].split("\t")
    mynode.append(int(d[0]))
    mynode.append(int(d[1]))
    i=i+1
#print(mynode)
node_ID1=np.unique(mynode)
node_ID=list(node_ID1)
#print(node_ID)
n=len(node_ID)
print(n)

2708


In [12]:
A=np.zeros((n,n))
i=0
for d in mylist:
    d=mylist[i].split("\t")
    A[node_ID.index(int(d[0]))][node_ID.index(int(d[1]))]=1
    i=i+1;
print(" The adjacency Matrix")
print("=======================")
print(A)

 The adjacency Matrix
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [55]:
D_only=[]
S_only=[]
Both=[]

In [239]:
p=random.sample(range(2708), 1000)
#print(p)
q=[]
for i in range(n):
    if i not in p:
        q.append(i)
#print(q)
class_60=np.array(catagories)
for d in p:
    class_60[d]=7
print(class_60)
for i in range(n):
    A[i][i]=class_60[i]

[1 7 4 ... 7 5 5]


In [240]:
Node_class=[0,1,2,3,4,5,6,7]
F_vec=spatial_two(Node_class, A,n)

Processing file 2707 (100%)

In [241]:
x =np.array(F_vec)
k=len(F_vec[0])
data=pd.DataFrame({"a_0":x[:,0]})
data.insert(loc=1, column="b_0", value=x[:,1])
for i in range(1,k-1,2):
    data.insert(loc=i+1, column="a_{}".format(int((i+1)/2)), value=x[:,i+1])
    data.insert(loc=i+2, column="b_{}".format(int((i+1)/2)), value=x[:,i+2])

data.insert(loc=k,column='Class',value=Data['class'].to_numpy())
data.head()

,a_0,b_0,a_1,b_1,a_2,b_2,a_3,b_3,a_4,b_4,...,b_11,a_12,b_12,a_13,b_13,a_14,b_14,a_15,b_15,Class
0,0,0,101,1,4,0,0,0,2,0,...,0,16,0,2,0,6,0,146,2,1
1,0,0,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,1,0,1,0,26,0,...,0,35,0,0,0,1,0,21,0,4
3,0,0,0,0,0,0,2,0,12,0,...,0,26,0,0,0,2,0,21,0,4
4,0,0,0,0,0,0,0,0,2,4,...,0,0,1,0,0,0,0,4,1,4


In [242]:
result = pd.concat([data,data1,data2], axis=1)
result.head()

,a_0,b_0,a_1,b_1,a_2,b_2,a_3,b_3,a_4,b_4,...,4,5,6,S_0,S_1,S_2,S_3,S_4,S_5,S_6
0,0,0,101,1,4,0,0,0,2,0,...,17,12,14,1,7,3,2,3,2,1
1,0,0,2,1,0,0,0,0,0,0,...,15,16,15,6,5,3,4,6,7,4
2,0,0,0,0,1,0,1,0,26,0,...,12,7,8,1,3,1,1,4,3,1
3,0,0,0,0,0,0,2,0,12,0,...,20,14,18,5,4,4,4,8,2,5
4,0,0,0,0,0,0,0,0,2,4,...,20,15,17,5,3,3,2,8,2,3


In [243]:
feature=["a_0","b_0"]
for i in range(1,k-1,2):
    feature.append("a_{}".format(int((i+1)/2)))
    feature.append("b_{}".format(int((i+1)/2)))
l=7
for i in range(l):
    feature.append("S_{}".format(i))
for i in range(l):
    feature.append("{}".format(i))

    
    
X=result[feature] # Features
y=result['Class']  # Labels
X_train=X.iloc[q]
X_test=X.iloc[p]
y_train=y.iloc[q]
y_test=y.iloc[p]

from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
# apply same transformation to test data
X_test = scaler.transform(X_test)  

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=1000, warm_start=True)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred)*100,"%")
#Both.append(metrics.accuracy_score(y_test, y_pred)*100)

Accuracy: 87.0 %


In [244]:
class_60=np.array(catagories)
i=0
for d in p:
    class_60[d]=y_pred[i]
    i=i+1
#print(class_60)
for i in range(n):
    A[i][i]=class_60[i]
#print(A)
Node_class=[0,1,2,3,4,5,6]
F_vec=spatial_two(Node_class, A,n)

Processing file 2707 (100%)

In [245]:
x =np.array(F_vec)
k=len(F_vec[0])
data=pd.DataFrame({"a_0":x[:,0]})
data.insert(loc=1, column="b_0", value=x[:,1])
for i in range(1,k-1,2):
    data.insert(loc=i+1, column="a_{}".format(int((i+1)/2)), value=x[:,i+1])
    data.insert(loc=i+2, column="b_{}".format(int((i+1)/2)), value=x[:,i+2])

data.insert(loc=k,column='Class',value=Data['class'].to_numpy())
data.head()

,a_0,b_0,a_1,b_1,a_2,b_2,a_3,b_3,a_4,b_4,...,b_9,a_10,b_10,a_11,b_11,a_12,b_12,a_13,b_13,Class
0,2,0,154,3,5,0,0,0,3,0,...,0,0,0,23,0,4,0,7,0,1
1,0,0,3,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,1,0,1,0,40,0,...,0,0,0,53,0,0,0,1,0,4
3,0,0,0,0,0,0,3,0,14,0,...,0,3,0,44,0,1,0,2,0,4
4,0,0,0,0,0,0,0,0,4,4,...,0,0,0,3,2,0,0,0,0,4


In [246]:
result = pd.concat([data,data1,data2], axis=1)
result.head()

,a_0,b_0,a_1,b_1,a_2,b_2,a_3,b_3,a_4,b_4,...,4,5,6,S_0,S_1,S_2,S_3,S_4,S_5,S_6
0,2,0,154,3,5,0,0,0,3,0,...,17,12,14,1,7,3,2,3,2,1
1,0,0,3,1,0,0,0,0,0,0,...,15,16,15,6,5,3,4,6,7,4
2,0,0,0,0,1,0,1,0,40,0,...,12,7,8,1,3,1,1,4,3,1
3,0,0,0,0,0,0,3,0,14,0,...,20,14,18,5,4,4,4,8,2,5
4,0,0,0,0,0,0,0,0,4,4,...,20,15,17,5,3,3,2,8,2,3


In [247]:
feature=["a_0","b_0"]
for i in range(1,k-1,2):
    feature.append("a_{}".format(int((i+1)/2)))
    feature.append("b_{}".format(int((i+1)/2)))
l=7
for i in range(l):
    feature.append("S_{}".format(i))
for i in range(l):
    feature.append("{}".format(i))

    
    
X=result[feature] # Features
y=result['Class']  # Labels
X_train=X.iloc[q]
X_test=X.iloc[p]
y_train=y.iloc[q]
y_test=y.iloc[p]

from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
# apply same transformation to test data
X_test = scaler.transform(X_test)  

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=1000, warm_start=True)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred)*100,"%")
Both.append(metrics.accuracy_score(y_test, y_pred)*100)

Accuracy: 88.6 %


In [234]:
from xgboost import XGBClassifier

bst = XGBClassifier(n_estimators=100, max_depth=3, learning_rate=0.1)
bst.fit(X_train,y_train)
y_pred=bst.predict(X_test)
#print(y_pred)
#print(y_test)
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred)*100,"%")

Accuracy: 86.9 %


In [248]:
feature=["a_0","b_0"]
for i in range(1,k-1,2):
    feature.append("a_{}".format(int((i+1)/2)))
    feature.append("b_{}".format(int((i+1)/2)))
#l=7
#for i in range(l):
#    feature.append("S_{}".format(i))
#for i in range(l):
#    feature.append("{}".format(i))

    
    
X=result[feature] # Features
y=result['Class']  # Labels
X_train=X.iloc[q]
X_test=X.iloc[p]
y_train=y.iloc[q]
y_test=y.iloc[p]

from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
# apply same transformation to test data
X_test = scaler.transform(X_test)  

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred)*100,"%")
S_only.append(metrics.accuracy_score(y_test, y_pred)*100)

Accuracy: 83.3 %


In [249]:
feature=[]
#for i in range(1,k-1,2):
#    feature.append("a_{}".format(int((i+1)/2)))
#    feature.append("b_{}".format(int((i+1)/2)))
l=7
for i in range(l):
    feature.append("S_{}".format(i))
for i in range(l):
    feature.append("{}".format(i))

    
    
X=result[feature] # Features
y=result['Class']  # Labels
X_train=X.iloc[q]
X_test=X.iloc[p]
y_train=y.iloc[q]
y_test=y.iloc[p]

from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
# apply same transformation to test data
X_test = scaler.transform(X_test)  

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred)*100,"%")
D_only.append(metrics.accuracy_score(y_test, y_pred)*100)

Accuracy: 83.1 %


In [250]:
print(Both)
print(S_only)
print(D_only)

[89.1, 87.9, 88.2, 87.8, 88.4, 88.3, 88.8, 87.9, 88.4, 88.6]
[84.7, 81.2, 84.1, 81.39999999999999, 82.69999999999999, 83.3, 83.2, 83.2, 83.0, 83.3]
[81.5, 80.4, 83.1, 79.0, 80.30000000000001, 81.10000000000001, 85.6, 81.8, 82.39999999999999, 83.1]


In [217]:
Both.pop(-2)

87.5

In [251]:
print(statistics.mean(Both))
print(statistics.stdev(Both))
print(statistics.mean(S_only))
print(statistics.stdev(S_only))
print(statistics.mean(D_only))
print(statistics.stdev(D_only))

88.34
0.41686661868969016
83.00999999999999
1.067135313715079
81.83
1.8499549544065437
